In [ ]:
!pip uninstall -y openai openai-secret-manager -q
!pip install --no-cache-dir "openai==1.56.1" "httpx>=0.27,<0.28" "pydantic>=2.7,<3" requests -q


import os, sys
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os

for k in ["HTTP_PROXY","HTTPS_PROXY","ALL_PROXY","http_proxy","https_proxy","all_proxy","OPENAI_HTTP_PROXY"]:
    os.environ.pop(k, None)

from getpass import getpass
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass("Informe sua OPENAI_API_KEY: ")

from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

MODEL_CHAT = "gpt-3.5-turbo"
MODEL_STT  = "whisper-1"

import openai, inspect
print("openai version:", openai.__version__)
print("client ok:", isinstance(client, OpenAI))


Informe sua OPENAI_API_KEY: ··········
openai version: 1.56.1
client ok: True


In [ ]:
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional

class PatientIdentification(BaseModel):
    name: Optional[str] = Field(None, description="Nome do paciente, se houver")
    age: Optional[int] = Field(None, description="Idade, se informada")
    id_number: Optional[str] = Field(None, description="Documento/ID, se houver")

class MedicalExtraction(BaseModel):
    symptoms: List[str] = Field(default_factory=list, description="Lista de sintomas")
    patient_identification: PatientIdentification = Field(default_factory=PatientIdentification)
    reason_for_consultation: Optional[str] = Field(None, description="Motivo breve da consulta")

class MedicalPlan(BaseModel):
    diagnosis: str
    treatment_plan: str
    recommendations: str


In [ ]:
import requests, tempfile, uuid, re, json
from IPython.display import Audio, display

def download_audio_to_tmp(audio_url: str) -> str:
    r = requests.get(audio_url, stream=True, timeout=60)
    r.raise_for_status()
    suffix = ".mp3"
    for ext in [".mp3",".wav",".m4a",".mp4",".webm",".ogg"]:
        if audio_url.lower().endswith(ext):
            suffix = ext
            break
    fn = os.path.join(tempfile.gettempdir(), f"{uuid.uuid4().hex}{suffix}")
    with open(fn, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return fn


In [ ]:
def transcribe_audio(audio_path: str) -> str:
    with open(audio_path, "rb") as f:
        result = client.audio.transcriptions.create(
            model=MODEL_STT,
            file=f,
            # language="pt"  # opcional: force PT-BR
        )
    return result.text.strip()


In [ ]:
EXTRACTION_SYSTEM = """Você extrai informações médicas ESTRUTURADAS de um texto livre.
Retorne APENAS um JSON VÁLIDO. Campos:
- symptoms: lista de strings
- patient_identification: objeto { name?, age?, id_number? }
- reason_for_consultation: string curta
"""

EXTRACTION_USER_TEMPLATE = """Texto do paciente (PT-BR):
\"\"\"{input_text}\"\"\"

Retorne SOMENTE um JSON com o formato:
{{
  "symptoms": ["..."],
  "patient_identification": {{
     "name": "...",
     "age": 30,
     "id_number": "..."
  }},
  "reason_for_consultation": "..."
}}
"""

def call_chat_raw(messages, **kwargs):

    return client.chat.completions.create(
        model=MODEL_CHAT,
        messages=messages,
        temperature=0,
        **kwargs
    ).choices[0].message.content

def try_parse_json(text: str):
    try:
        return json.loads(text)
    except:

        m = re.search(r"\{[\s\S]*\}", text)
        if not m:
            raise
        candidate = m.group(0)
        candidate = candidate.replace("\n", " ").strip()
        candidate = re.sub(r",\s*}", "}", candidate)
        candidate = re.sub(r",\s*]", "]", candidate)
        return json.loads(candidate)

def extract_medical_info(free_text: str) -> MedicalExtraction:
    prompt = EXTRACTION_USER_TEMPLATE.format(input_text=free_text)
    raw = call_chat_raw([
        {"role":"system","content":EXTRACTION_SYSTEM},
        {"role":"user","content":prompt}
    ])
    data = try_parse_json(raw)
    try:
        return MedicalExtraction.model_validate(data)
    except ValidationError:

        data.setdefault("symptoms", [])
        data.setdefault("patient_identification", {})
        data.setdefault("reason_for_consultation", None)
        return MedicalExtraction.model_validate(data)


In [ ]:
DIAGNOSIS_SYSTEM = (
    "Você é um assistente médico. Receberá um JSON já estruturado. "
    "Gere três partes claras e responsáveis: diagnóstico, plano de tratamento e recomendações. "
    "Não invente dados. Responda em PT-BR."
)

DIAGNOSIS_USER_TEMPLATE = (
    "Dados do paciente (JSON):\n"
    "{json_payload}\n\n"
    "Responda exatamente nos três blocos:\n"
    "1) Diagnóstico:\n"
    "2) Plano de tratamento:\n"
    "3) Recomendações:\n"
)

def generate_medical_plan(extraction: MedicalExtraction) -> MedicalPlan:
    import re, json
    payload = extraction.model_dump()
    prompt = DIAGNOSIS_USER_TEMPLATE.format(
        json_payload=json.dumps(payload, ensure_ascii=False, indent=2)
    )

    text = call_chat_raw(
        [
            {"role": "system", "content": DIAGNOSIS_SYSTEM},
            {"role": "user", "content": prompt},
        ]
    ).strip()

    diag = plan = recs = ""
    m1 = re.search(r"(?s)1\)\s*Diagnóstico:\s*(.*?)(?=2\)|$)", text)
    m2 = re.search(r"(?s)2\)\s*Plano de tratamento:\s*(.*?)(?=3\)|$)", text)
    m3 = re.search(r"(?s)3\)\s*Recomendações:\s*(.*)$", text)
    if m1: diag = m1.group(1).strip()
    if m2: plan = m2.group(1).strip()
    if m3: recs = m3.group(1).strip()
    if not (diag and plan and recs):

        diag, plan, recs = text, "", ""
    return MedicalPlan(diagnosis=diag, treatment_plan=plan, recommendations=recs)


In [ ]:
texto_fonte = "Sou a Maria, 64 anos. Tenho dor de cabeça há dois dias, tontura e enjoo."
extracted = extract_medical_info(texto_fonte)


plan = generate_medical_plan(extracted)
print("\n=== DIAGNÓSTICO ===\n", plan.diagnosis)
print("\n=== PLANO DE TRATAMENTO ===\n", plan.treatment_plan)
print("\n=== RECOMENDAÇÕES ===\n", plan.recommendations)


=== DIAGNÓSTICO ===
 Com base nos sintomas relatados pela paciente Maria, de dor de cabeça, tontura e enjoo há dois dias, é possível suspeitar de uma enxaqueca com sintomas vestibulares. No entanto, é importante realizar uma avaliação médica detalhada para confirmar o diagnóstico.

=== PLANO DE TRATAMENTO ===
 O tratamento para enxaqueca com sintomas vestibulares pode incluir o uso de medicamentos para alívio da dor de cabeça e enjoo, além de medidas para controle da tontura, como repouso, hidratação adequada e evitar ambientes com estímulos visuais intensos. Em casos mais graves, pode ser necessário o acompanhamento de um neurologista para orientações específicas.

=== RECOMENDAÇÕES ===
 Recomenda-se que a paciente Maria procure um médico para uma avaliação completa e adequada, a fim de confirmar o diagnóstico e receber o tratamento adequado. Além disso, é importante manter um registro dos sintomas, evitar situações desencadeantes conhecidas e adotar hábitos saudáveis de sono, alimen

In [ ]:

from google.colab import files
up = files.upload()
local_audio_path = next(iter(up))

from IPython.display import Audio, display
display(Audio(filename=local_audio_path))

transcribed = transcribe_audio(local_audio_path)
print("\n--- TRANSCRIÇÃO ---\n", transcribed)

extracted = extract_medical_info(transcribed)

import json
print("\n--- EXTRAÇÃO (JSON) ---\n" +
      json.dumps(extracted.model_dump(), ensure_ascii=False, indent=2))

plan = generate_medical_plan(extracted)
print("\n=== DIAGNÓSTICO ===\n", plan.diagnosis)
print("\n=== PLANO DE TRATAMENTO ===\n", plan.treatment_plan)
print("\n=== RECOMENDAÇÕES ===\n", plan.recommendations)


Saving reclama.mp3 to reclama (1).mp3



--- TRANSCRIÇÃO ---
 Eu sou o João, tenho 36 anos, tenho febre e dor de cabeça há 4 dias, tontura e enjoo. Vim porque hoje ficou muito pior.

--- EXTRAÇÃO (JSON) ---
{
  "symptoms": [
    "febre",
    "dor de cabeça",
    "tontura",
    "enjoo"
  ],
  "patient_identification": {
    "name": "João",
    "age": 36,
    "id_number": null
  },
  "reason_for_consultation": "Piora dos sintomas de febre, dor de cabeça, tontura e enjoo"
}

=== DIAGNÓSTICO ===
 Com base nos sintomas relatados pelo paciente João (36 anos), o diagnóstico provável é de uma infecção viral, como uma gripe ou virose, devido à presença de febre, dor de cabeça, tontura e enjoo.

=== PLANO DE TRATAMENTO ===
 Recomenda-se repouso, hidratação adequada, alimentação leve e balanceada, além do uso de medicamentos para controle da febre e dos sintomas, como analgésicos e antitérmicos. Caso os sintomas persistam ou piorem, é importante retornar para reavaliação médica.

=== RECOMENDAÇÕES ===
 - Descansar e evitar atividades f